## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/ivan/Desktop/Bootcamp/Module 6/world_weather_analysis/weather_database/weatherpy_database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,-9.8000,-139.0333,78.08,74,0,16.82,Clear
1,1,Mattru,SL,7.6244,-11.8332,93.20,36,14,5.41,Clouds
2,2,Ketchikan,US,55.3422,-131.6461,27.07,80,90,3.44,Clouds
3,3,Guerande,FR,47.3291,-2.4283,58.69,90,82,8.32,Clouds
4,4,Bluff,NZ,-46.6000,168.3333,57.67,87,99,15.88,Clouds
5,5,Chardara,KZ,41.2547,67.9692,48.58,35,99,7.47,Clouds
6,6,Hermanus,ZA,-34.4187,19.2345,65.59,74,63,17.90,Clouds
7,7,Mataura,NZ,-46.1927,168.8643,56.39,92,89,4.76,Clouds
8,8,San Vicente,AR,-26.6167,-54.1333,93.15,24,12,2.86,Clouds
9,9,Albany,US,42.6001,-73.9662,46.98,86,95,4.65,Clouds


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,-9.8000,-139.0333,78.08,74,0,16.82,Clear
10,10,Vaini,TO,-21.2000,-175.2000,75.36,94,90,0.00,Rain
12,12,Puerto Ayora,EC,-0.7393,-90.3518,80.56,75,92,9.91,Clouds
15,15,Inhapim,BR,-19.5492,-42.1200,81.19,83,100,7.65,Clouds
17,17,Tecoanapa,MX,16.5167,-98.7500,80.73,56,2,5.79,Clear
27,27,Rikitea,PF,-23.1203,-134.9692,77.74,72,45,12.53,Clouds
29,29,Matara,LK,5.9485,80.5353,76.98,84,72,10.98,Clouds
33,33,Kapaa,US,22.0752,-159.3190,75.18,100,90,8.05,Rain
37,37,Yulara,AU,-25.2406,130.9889,75.04,41,52,8.05,Clouds
40,40,Hithadhoo,MV,-0.6000,73.0833,81.68,73,5,9.73,Clear


In [32]:
# 4a. Determine if there are any empty rows.
#preferred_cities_df.sum().isna()
preferred_cities_df.count()

City_ID                524
City                   524
Country                524
Lat                    524
Lng                    524
Max Temp               524
Humidity               524
Cloudiness             524
Wind Speed             524
Current Description    524
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,78.08,Clear,-9.8000,-139.0333,
10,Vaini,TO,75.36,Rain,-21.2000,-175.2000,
12,Puerto Ayora,EC,80.56,Clouds,-0.7393,-90.3518,
15,Inhapim,BR,81.19,Clouds,-19.5492,-42.1200,
17,Tecoanapa,MX,80.73,Clear,16.5167,-98.7500,
27,Rikitea,PF,77.74,Clouds,-23.1203,-134.9692,
29,Matara,LK,76.98,Clouds,5.9485,80.5353,
33,Kapaa,US,75.18,Rain,22.0752,-159.3190,
37,Yulara,AU,75.04,Clouds,-25.2406,130.9889,
40,Hithadhoo,MV,81.68,Clear,-0.6000,73.0833,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [26]:
hotel_df.head(100)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,78.08,Clear,-9.8000,-139.0333,Villa Enata
10,Vaini,TO,75.36,Rain,-21.2000,-175.2000,Keleti Beach Resort
12,Puerto Ayora,EC,80.56,Clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
15,Inhapim,BR,81.19,Clouds,-19.5492,-42.1200,hotel Carreiro em Inhapim/MG
17,Tecoanapa,MX,80.73,Clear,16.5167,-98.7500,Hotel Manglares
...,...,...,...,...,...,...,...
365,La Orilla,MX,81.27,Clear,17.9833,-102.2333,Baymont by Wyndham Lazaro Cardenas
369,Karratha,AU,81.63,Clouds,-20.7377,116.8463,ibis Styles Karratha
373,Fare,PF,81.00,Clouds,-16.7000,-151.0167,Maitai Lapita Village Huahine
377,Thinadhoo,MV,80.83,Clouds,0.5333,72.9333,


In [31]:
# 7. Drop the rows where there is no Hotel Name.

hotel_df.count()

City                   524
Country                524
Max Temp               524
Current Description    524
Lat                    524
Lng                    524
Hotel Name             524
dtype: int64

In [35]:
# 8a. Create the output File (CSV)
output_data_file = "/Users/ivan/Desktop/Bootcamp/Module 6/world_weather_analysis/vacation_search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [39]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [41]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))